In [286]:
import numpy as np
import random

# Load data

In [287]:
with open("../data/dog_names.txt", "r") as f:
    names = list(set([i for i in f.read().splitlines()]))

In [288]:
names_train = random.sample(names, k=int(0.95*(len(names))))
names_set = list(set(names) - set(names_train))

# Process data

In [418]:
def ngram_list(list_object, n):
    ngrams = []
    for element in list_object:
        if len(element)>=n:
            ngrams.append(("<>" + element[0:n-2], element[n-2]))
            for idx in range(len(element) - n + 1):
                ngrams.append((element[idx:idx+n-1], element[idx+n-1]))
            ngrams.append((element[len(element)-n+1:], "<>"))
    return ngrams

def ngram_list_updated(list_object, n):
    ngrams = []
    for n_of_grams in range(2, n+1):
        for element in list_object:
            element = ["<>"] + list(element) + ["<>"]
            if len(element)>=(n_of_grams-2):
                for idx in range(len(element) - n_of_grams + 1):
                    ngrams.append(("".join(element[idx:idx+n_of_grams-1]), "".join(element[idx+n_of_grams-1])))
    return ngrams

def ngram_count(list_ngrams):
    ngrams_counts = {}
    for ngram in list_ngrams:
        if ngram in ngrams_counts:
            ngrams_counts[ngram] += 1
        else:
            ngrams_counts[ngram] = 1
    return ngrams_counts

def calculate_conditional_probabilities(ngram_counts):
    firsts = sorted(list(set([i[0] for i in ngram_counts.keys()])))
    nexts = sorted(list(set([i[1] for i in ngram_counts.keys()])))
    probabilities = np.zeros((len(firsts), len(nexts)))
    for idx_f, f in enumerate(firsts):
        for idx_n, n in enumerate(nexts):
            probabilities[(idx_f, idx_n)] = counts.get((f, n), 0)
    probabilities = probabilities / probabilities.sum(axis=1, keepdims=True)
    return firsts, nexts, probabilities

def generate_word(firsts, nexts, probabilites, n):
    first_char = str(np.random.choice(nexts, size=1, replace=True, p=probabilities[firsts.index("<>")])[0])
    word = first_char
    while True:
        prev_ngram = word[-(n-1):] if len(word)>=(n-1) else '<>'+word
        next_char = str(np.random.choice(nexts, size=1, replace=True, p=probabilities[firsts.index(prev_ngram)])[0])
        if next_char == "<>":
            break
        word += next_char
    return word

def generate_words(n_words, firsts, nexts, probabilities, n):
    words = []
    for i in range(n_words):
        words.append(generate_word(firsts, nexts, probabilities, n))
    return words

def calculate_perplexity(word, probabilities, n):
    word = ["<>"] + list(word) + ["<>"]
    predictor_grams = []
    for idx_char, char in enumerate(word[:-1]):
        predictor_grams.append("".join(word[max(0, idx_char - (n-1) + 1):idx_char+1]))
    perplexity = 1
    for predictor, test in zip(predictor_grams, word[1:]):
        try:
            probability = float(probabilities[firsts.index(predictor)][nexts.index(test)])
            probability = probability if probability>0 else 0.001
        except:
            probability = 1
        perplexity *= (probability)**(-1/len(predictor_grams))
    return perplexity

def calculate_test_set_perplexities(test_list, probabilities, n):
    perplexities = []
    for element in test_list:
        perplexities.append(calculate_perplexity(element, probabilities, n))
    return perplexities

In [429]:
n = 5
ngrams = ngram_list_updated(names_train, n)
counts= ngram_count(ngrams)
firsts, nexts, probabilities = calculate_conditional_probabilities(counts)
words = generate_words(1000, firsts, nexts, probabilities, n)
perplexities = calculate_test_set_perplexities(names_test, probabilities, n)

In [430]:
sum(perplexities)

2762.8904731790235

In [431]:
len(set(words).intersection(set(names_train)))/len(set(words))

0.6805845511482255

In [432]:
set(words) - set(names_train)

{'achilin',
 'adaschan',
 'adellek',
 'admiranhatty',
 'adventu',
 'airi',
 'ajos',
 'alado',
 'aladonny',
 'alettylos',
 'alfa',
 'alino',
 'alis',
 'amely',
 'amero',
 'anacho',
 'anessinja',
 'anti',
 'aphrates',
 'aquarry',
 'aranit',
 'ardo',
 'areni',
 'arica',
 'aristy',
 'arnaby',
 'arnet',
 'artie',
 'asha',
 'asheresa',
 'asmar',
 'assios',
 'auringo',
 'ayukarin',
 'babetty',
 'baccaramba',
 'bacimar',
 'baffi',
 'bagney',
 'balerina',
 'banglei',
 'basugar',
 'bikola',
 'boomeron',
 'bootsie',
 'branke',
 'braxas',
 'cagnan',
 'calet',
 'camilena',
 'campire',
 'candia',
 'caralie',
 'carrel',
 'cassima',
 'chakom',
 'chamberlanagan',
 'charlon',
 'charlyn',
 'charontus',
 'chestina',
 'chiccos',
 'chie',
 'chino',
 'chocoletto',
 'chubble',
 'ciester',
 'cintill',
 'corally',
 'cordono',
 'coustaaf',
 'cress',
 'crick',
 'daphire',
 'desper',
 'diamarcon',
 'disantus',
 'divanna',
 'domington',
 'dono',
 'dore',
 'dreamweavera',
 'egilia',
 'ellini',
 'elody',
 'emera',
 '